## Machine Learning Summer School 2021 - Reinforcement Learning Workshop
# Assignment 3 (Optional): [DDPG](https://arxiv.org/pdf/1509.02971.pdf)

Welcome to DDPG part of the workshop! :) In this assignment we aim to show you how continous action spaces can be handled using [CONTINUOUS CONTROL WITH DEEP REINFORCEMENT LEARNING](https://arxiv.org/pdf/1509.02971.pdf) by Lillicrap et al. This assignment will be best understood if *Assignment 2: DQN* is completed, as DDPG can be considered a natural extension to DQN for continous action spaces. This notebook is divided into three main sections: 

1. **Problem description**: Where we introduce the Pendulum problem which we'll solve using the DDPG paper.
2. **DQN implementation**: Where we explain our implementation of the DDPG paper, and task you with comleting that implementation in two exercises.
3. **Experimentation**: Where we give notes on how to tune the DDPG algorithm hyper-parameters and ask you to analyze results; and provide some closure notes.

**NOTE!**: Certain portion of the code is hidden, and only imported into this notebook. You can see full code base by clicking the Jupyter logo in the upper left corner.

## Problem description

We will test DDPG on [The Pendulum Problem](https://gym.openai.com/envs/Pendulum-v0/). The pendulum starts in random position hanging from its pivot point. The agent's goal is to prevent the pendulum from reaching its neutral equilibrium position and to keep it upright as long as possible. Additionally, the agent is rewarded for using the least amount of effort. The optimal policy would, thus, be to find and keep the [meta-stable equilibrium position](https://phys.libretexts.org/Bookshelves/Conceptual_Physics/Book%3A_Body_Physics_-_Motion_to_Metabolism_(Davis)/05%3A_Maintaining_Balance/5.06%3A_Types_of_Equilibrium) where agent requires least effort to keep the pendulum upright. The agent can affect the pendulum by applying force to it -- its action space constitues of a single real numbered value in range $[-2.0, 2.0]$. The agent's observation space constitutes of the angle the pendulum forms with the vertical axis, or more precisely its $\sin$ and $\cos$, and pendulum's angular velocity. In other words, the state-space has three dimensions of continues values and the action space has one dimension of continous values.

The video below shows an **untrained agent** trying to solve this problem. In comparison to the Cartpole problem, episodes do not terminate once the pendulum reaches its neutral equilibrium position, as agent can still move it upright with the right combination of actions. 


![cartpole](resources/pendulum_env.mp4)

If the video above won't load properly, run the cell below to display the video.

In [ ]:
import IPython
IPython.display.Video('./resources/pendulum_env.mp4', html_attributes="loop autoplay")

## DDPG implementation

### Intro

[DDPG](https://arxiv.org/pdf/1509.02971.pdf) can be considered an adaptation of DQN algorithm to continous action spaces. In order to understand the contribution of DDPG, we must first briefly explain why continous action spaces represent a problem for DQN (and TD algorithms in general). Consider the Q-value update rule that we've used in the DQN assignment:

$$y_j = r_j + (1 - \eta_{j+1})\gamma\max_{a'}\hat{Q}(s_{j+1}, a; \theta))$$

If the action space is continous, how can we perform the $max$ operation over actions? In fact, what is the Q-value function to output if we just pass it an input state $s$? It cannot output discrete Q-values for each action, as actions aren't discrete, and it cannot output a single value because we won't know which action this output refers to.

DDPG addresses this issue with two main design changes:

* It constructs the Q-value network so that it accepts at input the concatenation of state and action, and outputs a single value corresponding to the Q-value of that particular state-action pair.
* It uses an additional network $\mu(a|s)$ which is optimized to estimate the **optimal** action $a$ an agent can take at state $s$. This policy network takes a state at input, and outputs a continous value representing the action to be taken at that state.

With these two design changes, we can rewrite the Bellman update from the previous equation as:

$$y_j = r_j + (1 - \eta_{j+1})\gamma\hat{Q}(s_{j+1}, \mu(a|s_{j+1}; \phi); \theta))$$

where we have just replaced the $\max_{a'}\hat{Q}(s_{j+1}, a; \theta)$ with $\hat{Q}(s_{j+1}, \mu(a|s_{j+1}; \phi); \theta)$. We can do this because we directly optimize $\mu(a|s; \phi)$ to produce the action that maximizes the Q-value at state $s$. This maximization can be done using gradient-ascent algorithm, using the same samples from the replay buffer we use to update the Q-network: 

$$\max_{\phi}\mathbb{E}_{s \sim D}\left[Q(s, \mu(a|s; \phi); \theta)\right]$$

In other words, we maximize the parameters $\phi$ of the policy network $\mu(a|s)$ with respect to the Q-value produced by the Q-value network at state $s$ and action $a = \mu(a|s)$ picked by the policy network.

Do not warry if this explanation of DDPG seems a bit unclear. It is highly condensed, and does not go into details and full motivation behind the DDPG algorithm. If you are interested to learn more, we highly recommend [Spinning Up](https://spinningup.openai.com/en/latest/algorithms/ddpg.html) explanation of DDPG which goes into greater detail.

If you have completed the DQN assignment, you will find that the solution to this one fundamentally differs in only couple of lines of code, which we will guide you through step by step. 

### Implementation

DDPG implementation is divided into five cells. We encourage you to read through all cells, but to only modify those specified as **Exercise 1: Action selection** and **Exercise 2: Q-value and Policy update**.

**NOTE!**: All cells need to be ran!

Run the cell below to import the required libraries. We use [PyTorch](https://pytorch.org) to implement the DQN algorithm, while [Gym](https://gym.openai.com) provides the Cartpole environment. File [nets.py](../edit/nets.py) implements the neural networks that will be used in this tutorilal, while [utils.py](../edit/utils.py) provides the neccessary utilities.

In [2]:
import os
import random
import torch
import time
import numpy as np
import gym

from env_wrapper import EnvWrapper
from nets import ContinuousQNet, Actor
from utils import create_run_name, visualize_result
from rb import ReplayBuffer

import jdc

random.seed(0)
STORE_PATH = './tmp_ddpg_learning'

In the cell below we define the DDPG initialization procedure. **state_size**, **action_size**, **action_limit**, **num_hidden**, **hidden_units** are used to construct the two neural  neural network types the DDPG agent will use. **state_size** and **action_size** will specify the input dimension to [ContinousQNet](../edit/nets.py) which outputs a single Q-value for the passed (state, action) pair. **state_size** and **action_limit** will be used to construct the [Actor](../edit/nets.py) network, which takes a state and produces a real valued action limited by **action_limit**, which is a property of the environment (in the case of Pendulum equals $[-2.0, 2.0]$). **num_hidden** and **hidden_units** specify the number of hidden layers and the number of units in each, respectively.

The rest of the constructor arguments are some of the hyper-parameters of the algorithm you will be able to play around with. **gamma** ($\gamma$) represents the discount factor, **batch_size** represents the number of samples in mini-batch $h$, while **lr** is the learning rate of optimizers used to update the two neural network types we've mentioned above. Finally, **eps_start** represents the start value of the exploration rate, which is used in a new way compared to the previous two assignments. We will explain this in greater detail down the line.

Finally, notice that we are constructing four neural networks in total. As with the case of DQN, we have **q_current** $Q(s,a; \theta)$ and **q_target** $\hat{Q}(s, a; \hat{\theta})$ networks that approximate Q-values. Additionally, we also construct the policy network **mu_current** $\mu(a|s; \phi)$, as well as its target network **mu_target** $\hat{\mu}(a|s; \hat{\phi})$. We use $\hat{}$ to dentote networks, and their respective parameters, that are not learned, but rather copied over from their *live* counterparts. Theese target networks play the same role as they did in the DQN assignment.

In [3]:
class DDPGAgent(object):
    def __init__(self, state_size, action_size, action_limit, gamma=0.95, batch_size=64, lr=0.00025,
                 num_hidden=2, hidden_units=32, eps_start=0.4):
        self.action_size = action_size
        self.state_size = state_size
        self.action_limit = action_limit
        self.gamma = gamma
        self.name = 'DDPG'

        self.q_current = ContinuousQNet(state_size, action_size, h=hidden_units, num_hidden=num_hidden)
        self.q_target = ContinuousQNet(state_size, action_size, h=hidden_units, num_hidden=num_hidden)
        for p in self.q_target.parameters():
            p.requires_grad = False
        self.q_target.load_state_dict(self.q_current.state_dict())

        self.mu_current = Actor(state_size, action_size, action_limit, h=hidden_units, num_hidden=num_hidden)
        self.mu_target = Actor(state_size, action_size, action_limit, h=hidden_units, num_hidden=num_hidden)
        for p in self.mu_target.parameters():
            p.requires_grad = False
        self.mu_target.load_state_dict(self.mu_current.state_dict())

        self.rb = self.init_rb()
        self.batch_size = batch_size

        self.epsilon = eps_start

        q_learning_rate = lr
        mu_learning_rate = lr
        self.q_optimizer = torch.optim.Adam(self.q_current.parameters(), lr=q_learning_rate)
        self.mu_optimizer = torch.optim.Adam(self.mu_current.parameters(), lr=mu_learning_rate)
        self.polyak = 0.995

The next cell specifies DDPGAgent methods that deal with the replay memory and experience replay. **init_rb(self)** constructs the replay buffer $D$ which is used to store transitions. **remember(self, state, action, reward, next_state, done)** stores the transition $(s_j, a_j, s_{j+1}, r_j, \eta_{j+1})$ to the replay buffer, while **sample(self)** samples minibatch of size DDPGAgent.batch_size ($h$) from it. This is the exact same code we've used in the DQN assignment. If you want to check the specifics of our implementation of replay buffer, you can find it in [rb.py](../edit/rb.py).

In [4]:
%%add_to DDPGAgent
def init_rb(self):
    # Replay buffer initialization.
    replay_buffer = ReplayBuffer(1e5, obs_dtype=np.float32, act_dtype=np.float32, default_dtype=np.float32)
    return replay_buffer

def remember(self, state, action, reward, next_state, done):
    # Remember (Q,S,A,R,S') as well as whether S' is terminating state.
    self.rb.add(obs=state, act=action, rew=reward, next_obs=next_state, done=done)

def sample(self):
    states, actions, next_states, rewards, dones = self.rb.sample(self.batch_size)
    return states, actions, rewards, next_states, dones

In the DQN assignment we've stated that there are multiple ways to periodically update the weights of *target* networks. DQN, then, used a simple scheme where weights from *live* networks were copied into the *target* networks every $K$ steps. This approach is also known as a *hard update*. In contrast, the DDPG paper uses a *soft update* approach, where weights in *target* networks are updated every step by small amount by *Polyak averaging*:

$$\hat{\theta} \leftarrow \rho\hat{\theta} + (1-\rho)\theta$$
$$\hat{\phi} \leftarrow \rho\hat{\phi} + (1-\rho)\phi$$

Parameter $\rho$ is a hyper-parameter of the DDPG algorithm, and is set to **DDPGAgent.polyak=0.995**. Setting it to 0 would produce *hard updates* as are used in the DQN, but that would be highly problematic in this scenario because **\_update_target_model(self)** is called in *every* update step, that is, in *every* call to **DDPGAgent.backward()**. 

In [5]:
%%add_to DDPGAgent
def _update_target_model(self):
    with torch.no_grad():
        for q_net, q_targ in zip(self.q_current.parameters(), self.q_target.parameters()):
            q_targ.data.mul_(self.polyak)
            q_targ.data.add_((1 - self.polyak) * q_net.data)
        for mu_net, mu_targ in zip(self.mu_current.parameters(), self.mu_target.parameters()):
            mu_targ.data.mul_(self.polyak)
            mu_targ.data.add_((1 - self.polyak) * mu_net.data)

#### Exercise 1: Action selection

Complete the function **select_action(self, state)** below to greedily select best actions. **We strongly encourage you to read through the rest of the code explanations before starting the exercise.**

Notice how we use exploration in DDPG. Recall that in assignments 1 & 2 we've used some small value of $\varepsilon$ to select random action, or act optimally otherwise. DDPG explores in a slightly different way. Namely, instead of selecting completely random action with probability $\varepsilon$, we add some small noise in the form of standard normal distribution to the selected optimal continuous action. This random noise is scaled by $\varepsilon$, which decreases during training. At test time, however, no noise would be added to selected actions. 

Constructor argument **eps_start** represents the initial scale of this random noise added to the action. We anneal it linearly after each epsiode, over some initial number of episodes after which it is kept constant.

In [38]:
%%add_to DDPGAgent
def select_action(self, state):
    # ######## TODO: IMPLEMENT DDPG ACTION SELECTION HERE ########
    # Hint 1: We cannot use argmax over Q values in the continuous
    # action space case.
    # Hint 2: There are two network types at play in DDPG!
    action = None
    
    
    action += self.epsilon * np.random.randn(self.action_size)
    return np.clip(action, -self.action_limit, self.action_limit)

#### Exercise 2: Q-value and policy update

Complete the function **backward(self)** below to update current estimates of Q-values $Q(s, a; \theta)$ and policy $\mu(a|s; \phi)$. 

The Q-value estimate update is very similar to the DQN update, and you can copy the code from the previous assignment and use it as a starting point. Remember to use *target* networks when calculating the Bellman update! You can use the equation below:

$$y_j = r_j + (1 - \eta_{j+1})\gamma\hat{Q}(s_{j+1}, \hat{\mu}(a|s_{j+1}; \hat{\phi}); \hat{\theta}))$$

The policy network is updated using the gradient ascent algorithm with respect to Q-value estimation at current state sampled from the replay memory, and action selected by the policy network $\mu(a|s; \phi)$. You can use the equations below as a starting point:

$$\max_{\phi}\mathbb{E}_{s \sim D}\left[Q(s, \mu(a|s; \phi); \theta)\right]$$
$$\nabla_{\phi}\frac{1}{|H|}\sum_{s \in H}Q(s, \mu(a|s; \phi);\theta))$$

where $H$ is the mini-batch we've sampled from the replay memory, $|H|$ is the number of samples in the mini-batch, and $s$ are the current states from the mini-batch sample. The first equation specifies that gradient ascent is performed with respect to the expectation of Q-values of samples drawn from the replay memory $D$, while the second equation explicitely calculates this expectation with respect to the mini-batch.



**We strongly encourage you to read through the rest of the code explanations before starting the exercise.**

In [40]:
%%add_to DDPGAgent
def backward(self):
    # ######## TODO: IMPLEMENT Q-NETWORK UPDATE STEP HERE ###############
    # # 1. Sample mini-batch of stored transitions from the replay buffer

    

    # # 2. Implement the Q-network update step.
    # ### a. Use "states" (batch_dim x self.state_size) and "actions" (batch_dim x action_size)
    #     to get Q values (batch_dim x 1) of actions agent had played out in the environment.
    #     USE qs_selected VARIABLE TO STORE RESULT!
    qs_selected = None
    

    
    # ### b. Use "next_states" (batch_dim x self.state_size) to calculate the target value
    #     Q values obtained in a. should be regressed to.
    #     Hint 1: You cannot use max over Q values in this use case.
    #     Hint 2: Target network plays important part here!
    #     Hint 3: Pay attention to PyTorch gradient accumulation!
    #     Hint 4: You can use "dones" (batch_dim x 1) to check whether "next_states" are
    #     terminating!
    #     USE qs_target VARIABLE TO STORE RESULT!
    qs_target=None
    
    
        
    
    # Code below updates the "live" network self.q_current using the variables you have
    # calculated in TODO section above: qs_selected and qs_target. DO NOT MODIFY THIS CODE.
    with torch.no_grad():
        td_error = torch.abs(qs_target - qs_selected).detach()
    
    # We update the "live" network, self.q_current. First we zero out the optimizer gradients
    # and then we apply the update step using qs_selected and qs_target. 
    self.q_optimizer.zero_grad()
    loss = (torch.nn.functional.mse_loss(qs_selected, qs_target)).mean()
    loss.backward()
    self.q_optimizer.step()
    
    # We now need to update the policy network as well. First we set up q-network
    # params to not require gradients so PyTorch does not waste resources keeping
    # track of q-network grads.
    for p in self.q_current.parameters():
        p.requires_grad = False
    
    # ######## TODO: IMPLEMENT POLICY-NETWORK UPDATE STEP HERE ###############
    # Store result to q_mu variable!
    # Hint: We should use gradient ascent to update the policy network! This means
    # minus should be squeezed in somewhere, as PyTorch optimizers assume by default
    # that gradient descent is performed.
    q_mu = None
    

    
    # We update the "live" network, self.mu_current. First we zero out the optimizer gradients
    # and then we apply the update step using q_mu.
    self.mu_optimizer.zero_grad()
    loss_mu = q_mu.mean()
    loss_mu.backward()
    self.mu_optimizer.step()

    for p in self.q_current.parameters():
        p.requires_grad = True

    self._update_target_model()
    return torch.mean(td_error).item(), loss_mu.detach().item()

The cell below contains all of the tunable hyper-parameters of the DDPGAgent. Once you have a working implementation of the two exercises above, you can tune these hyper-parameters and see how they affect the training process.

The hyper-parameters are mostly the same as in the DQN assignment. The only difference is the configuration of the exploration rate of the DDPG agent, as it uses a sligthly modified exploration scheme. **EPSILON_START** now represents the initial value of $\varepsilon$ which is now used to scale the gaussian noise we add to the actions, while **DECAY_EPISODES** represents the number of episodes we anneal $\varepsilon$ over. 

In [36]:
# ### HYPER PARAMETERS ##########
# ### EPISODE ###################
# Number of episodes to train for
EPISODES = 150
# Number of steps per episode
STEPS = 1000

# ### TRAINING ##################
# Start exploration rate
# for eps greedy policy
EPSILON_START = 0.4
# End exploration rate
EPSILON_END = 0.1
# Number of episodes to decay epsilon
# linearly over
DECAY_EPISODES = 0.3 * EPISODES
# Discount rate
GAMMA = 0.95
# DQN live network update
# frequency in STEPS
UPDATE_FREQ = 1
# Learning rate
LR = 0.0001
# Batch size
BATCH_SIZE = 64

# ### NETWORK ARCHITECTURE ######
# Number of hidden layers
# of H units
NUM_H = 1
# Number of units in hidden
# layers
H = 128
# ### HYPER PARAMETERS END #######

run_name = create_run_name(
        alg='DDPG',
        env='pendulum',
        num_layers=NUM_H,
        hidden_dim=H,
        eps_start=EPSILON_START,
        eps_end=EPSILON_END,
        decay=(EPSILON_START - EPSILON_END) / DECAY_EPISODES,
        gamma=GAMMA,
        batch_size=BATCH_SIZE,
        lr=LR,
        num_ep=EPISODES,
        num_step=STEPS,
        updt_freq=UPDATE_FREQ,
        sw_freq=None
    )

The code in the cell below encapsulates the DDPGAgent and controls its training process according to the algorithm presented in the introductiory section. We start with constructing the environment **env**, and the DDPGAgent itself. We then continue with the implementation of the overall DDPG algorithm from the introductory section.

**NOTE!**: Remember to run all previous cells. The final output of the cell below will be a figure displaing returns (rewards obtained throughout episodes), and TD Errors and Policy loss per each step when DDPG agent was updated (each backward() pass). You can review theese figures by clicking on Jupyter logo in top left corner, and navigating to [**tmp_ddpg_learning** folder](./tmp_ddpg_learning), where figures are stored and named based on the algorithm hyper-parameters that have generated them.

In [ ]:
env = EnvWrapper(gym_env=gym.make('Pendulum-v0'), steps=STEPS)
# Initialize Q networks, mu (policy) networks, replay memory
agent = DDPGAgent(
    state_size=env.state_size(),
    action_size=env.action_size(),
    action_limit=env.action_limit(),
    gamma=GAMMA,
    batch_size=BATCH_SIZE,
    lr=LR,
    eps_start=EPSILON_START,
    num_hidden=NUM_H,
    hidden_units=H
)

epsilon_decay_step = (EPSILON_START - EPSILON_END) / DECAY_EPISODES
results = []
td_errors = []
mu_losses = []
start = time.time()
random.seed(0)

for episode in range(EPISODES):
    # Start game/episode
    state = env.reset()
    cum_rew = 0

    # Loop inside one game episode
    for t in range(STEPS):
        # Display the game. Comment bellow line in order to get faster training.
        env.render()
        
        # ########## EXERCISE 1: ACTION SELECTION ##########################
        action = agent.select_action(state=torch.from_numpy(state).detach())
        # ##################################################################

        next_state, reward, done = env.step(action)
        cum_rew += reward
        
        # Store transition to replay memory
        agent.remember(state=state, action=action, reward=reward, next_state=next_state, done=float(done))
        
        # Update Q-values and policy 
        if episode > 10 and (episode + t) % UPDATE_FREQ == 0:
            # ## EXERCISE 2: Q-VALUE AND POLICY UPDATE ##
            td_error, policy_loss = agent.backward()
            # ###########################################
            td_errors.append(td_error)
            mu_losses.append(policy_loss)

        if done or (t == STEPS - 1):
            if episode > 10:
                print("EPISODE: {0: <4}/{1: >4} | EPSILON: {2: <5.3f} | SCORE: {3: <7.1f}"
                      " | TD ERROR: {4: <5.2f} | POLICY LOSS {5: <6.3f} "
                      .format(episode + 1, EPISODES, agent.epsilon, cum_rew, td_error, policy_loss))
            else:
                print("EPISODE: {0: <4}/{1: >4} | EPSILON: {2: <5.3f} | SCORE: {3: <7.1f}"
                      " | WARMUP - NO TD ERROR, NO POLICY LOSS"
                      .format(episode + 1, EPISODES, agent.epsilon, cum_rew))
            results.append(cum_rew)
            if agent.epsilon > EPSILON_END:
                agent.epsilon -= epsilon_decay_step
            break

        state = next_state

end = time.time()
total_time = end - start

print()
print("TOTAL EPISODES: {0: <4} | TOTAL UPDATE STEPS: {1: <7} | TOTAL TIME [s]: {2: <7.2f}"
      .format(EPISODES, len(td_errors), total_time))
print("EP PER SECOND: {0: >10.6f}".format(total_time / EPISODES))
print("STEP PER SECOND: {0: >8.6f}".format(total_time / len(td_errors)))

fig = visualize_result(
    returns=results,
    td_errors=td_errors,
    policy_errors=mu_losses
)
fig.savefig(os.path.join(STORE_PATH, run_name + '.png'), dpi=400)

## Experimentation

The end goal of this exercise is for you to get the feel of applying RL concepts in practice. To that effect, try to figure out an answer to questions below:

1. Which hyper-parameters affect training stability and why?
2. Which hyper-parameters affect the potential to obtain high rewards?

Finally,you can try to solve other problems using DDPG, provided they have continous action space. You can find additional pre-made environments on the [OpenAI Gym website](https://gym.openai.com/envs/#classic_control). If you wish to try this notebook on other problems, simply substitute the environment name on line 1 of the cell above with the new one. (e.g. **Pendulum-v0** -> **MountainCarContinuous-v0**)

Additionally, we provide a couple of resources for further study:

* If you are interested in continous action space applications of RL (robotics) we recommend acquainting yourself with policy optimization algorithms. [Spinning Up](https://spinningup.openai.com/en/latest/algorithms/vpg.html) has an excellent explanation of the Vanilla Policy Gradient (REINFORCE), as well as an introduction to more modern algorithms such as [Trust Region Policy Optimization](https://arxiv.org/abs/1502.05477) by Schulman et al, and [Proximal Policy Optimization](https://arxiv.org/abs/1707.06347) by Schulman et al. again :D (the al. part differs).
* Couple of open-source implementations of DDPG (and other algorithms!) are: [Baselines](https://github.com/openai/baselines), [Spinning Up](https://github.com/openai/spinningup), [rllab](https://github.com/rll/rllab). 
* A great blog to read about various RL (and ML) papers: [Lil'Log](https://lilianweng.github.io/lil-log/).